In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastrl['dev']  # upgrade fastrl on colab

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab(): 
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON

# Async Testing
> Testing async environment execution in a separate notebook.

One of the most important things to have working with the datablock is to answer: 

    Can we run multiple envs, in multiple workers, using a torch Module, all loaded on cuda?
    
You can however there are some important things to use in your notebook.

> Important: This is only important if you are training with `num_workers>0` and `str(default_device())=='cuda'`. If one of these is not true, then feel free
to train your model the *normal* way.

Relevant links:

  [setting the `MKL_THREADING_LAYER` variable](https://github.com/pytorch/pytorch/issues/37377)
  
  [handling `set_start_method('spawn')`](https://stackoverflow.com/questions/48822463/how-to-use-pytorch-multiprocessing)
  
  [using `if __name__ == '__main__'`](https://github.com/pytorch/pytorch/issues/3492)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
from fastai.data.load import DataLoader,_FakeLoader,multiprocessing,_MultiProcessingDataLoaderIter

In [ ]:
from torch.utils.data._utils import worker as z

In [ ]:
z??

Type:        module
String form: <module 'torch.utils.data._utils.worker' from '/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py'>
File:        /opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py
Source:     
r""""Contains definitions of the methods used by the _BaseDataLoaderIter workers.

These **needs** to be in global scope since Py2 doesn't support serializing
static methods.
"""

import torch
import random
import os
from collections import namedtuple
from torch._six import queue
from torch._utils import ExceptionWrapper
from typing import Union
from . import signal_handling, MP_STATUS_CHECK_INTERVAL, IS_WINDOWS

if IS_WINDOWS:
    import ctypes
    from ctypes.wintypes import DWORD, BOOL, HANDLE

    # On Windows, the parent ID of the worker process remains unchanged when the manager process
    # is gone, and the only way to check it through OS is to let the worker have a process handle
    # of the manager and ask if the process stat

In [ ]:
from fastai.learner import Learner

In [ ]:
from fastai.test_utils import synth_learner

In [ ]:
synth_learner().show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, Recorder]
  Start Epoch Loop
     - before_epoch   : [Recorder]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder]
      Start Batch Loop
         - before_batch   : []
         - after_pred     : []
         - after_loss     : []
         - before_backward: []
         - before_step    : []
         - after_step     : []
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder]
      Start Batch Loop
         - **CBs same as train batch**: []
      End Batch Loop
    End Valid
     - after_cancel_validate: [Recorder]
     - after_validate : [Recorder]
  End Epoch Loop
   - after_cancel_epoch: []
   - after_epoch    : [Recorder]
End Fit
 - after_cancel_fit: []
 - after_fit      : []


In [ ]:
TestDataset??

Init signature: TestDataset(*args, **kwds)
Docstring:     
An abstract class representing a :class:`Dataset`.

All datasets that represent a map from keys to data samples should subclass
it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
data sample for a given key. Subclasses could also optionally overwrite
:meth:`__len__`, which is expected to return the size of the dataset by many
:class:`~torch.utils.data.Sampler` implementations and the default options
of :class:`~torch.utils.data.DataLoader`.

.. note::
  :class:`~torch.utils.data.DataLoader` by default constructs a index
  sampler that yields integral indices.  To make it work with a map-style
  dataset with non-integral indices/keys, a custom sampler must be provided.
Source:        
class TestDataset(Dataset):
    def __init__(self,policy,device='cpu'):
        self.policy=policy
        self.device=device
        self.policy.to(device=self.device)

        self.pids=mp.Queue()
        self.pids.put

In [ ]:
if __name__ == '__main__':
    import os
    os.environ['MKL_THREADING_LAYER']="GNU"
    
    import torch.multiprocessing as mp
    try:mp.set_start_method('spawn',force=True)
    except Exception:pass
    
    # Third party libs
    from fastai.torch_basics import *
    from fastai.data.all import *
    # Local modules
    from fastrl.data.block import *
    from torch.optim import Adam

    
    dqn=DQN().share_memory()
    opt=Adam(dqn.parameters(),lr=0.01)
    ds=TestDataset(dqn,device=default_device())
    
    dl=DataLoader(ds,num_workers=2,batch_size=5,persistent_workers=True)
    for _ in range(4):
        for x in dl:
            print(x)
            out=dqn.policy(x)+1
            target=torch.ones(out.shape)+1
            loss=nn.MSELoss()(out,target)
            opt.zero_grad()
            loss.backward()
            opt.step()
            print(loss)

    while not ds.pids.empty(): print(ds.pids.get(),end=' ')
    while not ds.envs.empty(): print(ds.envs.get(),end=' ')

tensor([[ 0.0441,  0.1664,  0.0482, -0.3158],
        [ 0.0238,  0.2068, -0.0211, -0.2508],
        [ 0.0053,  0.1724,  0.0384, -0.2331],
        [-0.0118,  0.2236,  0.0053, -0.2778],
        [ 0.0426,  0.1874, -0.0471, -0.2632]])
tensor(0.8844, grad_fn=<MseLossBackward>)
tensor([[ 0.0441,  0.1664,  0.0482, -0.3158],
        [ 0.0238,  0.2068, -0.0211, -0.2508],
        [ 0.0053,  0.1724,  0.0384, -0.2331],
        [-0.0118,  0.2236,  0.0053, -0.2778],
        [ 0.0426,  0.1874, -0.0471, -0.2632]])
tensor(0.8049, grad_fn=<MseLossBackward>)
tensor([[-0.0353,  0.1899,  0.0222, -0.3078],
        [ 0.0056,  0.1533,  0.0337, -0.3161],
        [ 0.0202,  0.1772, -0.0246, -0.3179],
        [ 0.0058,  0.1980,  0.0412, -0.2539],
        [-0.0148,  0.2275, -0.0444, -0.3056]])
tensor(0.7305, grad_fn=<MseLossBackward>)
tensor([[ 0.0441,  0.1664,  0.0482, -0.3158],
        [ 0.0238,  0.2068, -0.0211, -0.2508],
        [ 0.0053,  0.1724,  0.0384, -0.2331],
        [-0.0118,  0.2236,  0.0053, -0.2778

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab(): 
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_nbdev_extension.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
